# Working With the AMISR Vector Velocity Data Product
This tutorial will go over the AMISR Vector Velocity (vvels) data product and how to use vvels files to find resolved electric fields and plasma drift velocity vectors.  The algorithm used to produce the vector velocity data product was originally described in Heinselman and Nicolls [2016].

## Reading Vvels Files
The vector velocity data is stored in hdf5 files.  These can be viewed with any hdf5 file viewer, such as [HDFView](https://www.hdfgroup.org/downloads/hdfview/).  There are two main python packages for reading in hdf5 files, [PyTables](https://www.pytables.org/) and [h5py](https://www.h5py.org/).  In this tutorial, we choose to use h5py, but either package will work.  Below, we read in the parameters from test_vvels.h5 that will be used later in this tutorial.

In [32]:
import h5py

filename = 'test_vvels.h5'

with h5py.File(filename, 'r') as h5file:
    unixtime = h5file['Time']['UnixTime'][:]
    mlat = h5file['Magnetic']['MagneticLatitude'][:]
    mlon = h5file['Magnetic']['MagneticLongitude'][:]
    
    Velocity = h5file['Magnetic']['Velocity'][:]
    SigmaV = h5file['Magnetic']['SigmaV'][:]
    
    ElectricField = h5file['Magnetic']['ElectricField'][:]
    SigmaE = h5file['Magnetic']['SigmaE'][:]

### Time Group
The Time group contains arrays that describe the start and end time of each record in the file.  This table describes each array and gives its shape, assuming the total number of records in an experiment is Nrecords.

| HDF5 Array name | Shape | Description |
|-------|-------|---------------|
| UnixTime | Nrecords x 2 | Unix timestamp (seconds since midnight on January 1, 1970) for start and end of each integration period |
| Year | Nrecords x 2 | Integer year for start and end of each integration period |
| Month | Nrecords x 2 | Integer month for start and end of each integration period |
| Day | Nrecords x 2 | Integer day for start and end of each integration period |
| doy | Nrecords x 2 | Integer day of year for start and end of each integration period |
| dtime | Nrecords x 2 | Decimal hour for start and end of each integration period |
| MagneticLocalTimeSite | Nrecords x 2 | Magnetic local time of the radar site for start and end time of each integration period |

### Magnetic Group
The Magnetic group of the hdf5 files contains the components of the plasma drift velocity and electric field and the position of these vectors in Apex magnetic coordinates (Richmond, 1995; Laundal and Richmond, 2016).

| HDF5 Array name | Shape | Description |
|-------|-------|---------------|
| MagneticLatitude | Nbins | Apex magnetic latitude of each bin |
| MagneticLongitude | Nbins | Apex magnetic longitude of each bin |
| Velocity | Nrecords x Nbins x 3 | Apex components of plasma drift velocity, $v_{e_1}$, $v_{e_2}$, and $v_{e_3}$, for each bin at each time |
| SigmaV | Nrecords x Nbins x 3 x 3 | Plasma drift velocity covariance matrix for each bin at each time |
| ElectricField | Nrecords x Nbins x 3 | Apex components of convection electric field, $E_{d_1}$, $E_{d_2}$, and $E_{d_3}$, for each bin at each time |
| SigmaE | Nrecords x Nbins x 3 x 3 | Convection electric field covariance matrix for each bin at each time |
| Chi2 | Nrecords x Nbins | $\chi^2$ goodness of fit value for each bin at each time |

## Using Apex Components
The Apex coordinate system uses two different sets of base vector.
- $\bf{e_1}$, $\bf{e_2}$, $\bf{e_3}$ correspond roughly to the covariant base vectors $\bf{\epsilon_i}$ (Laundal and Richmond, 2016, eqn. 57-59)
- $\bf{d_1}$, $\bf{d_2}$, $\bf{d_3}$ correspond roughly to the contravarient base vectors $\bf{\epsilon^i}$ (Laundal and Richmond, 2016, eqn. 53-55)
The advantage to using Apex coordinates is that the components $v_{e_1}$, $v_{e_2}$, $v_{e_3}$ and $E_{d_1}$, $E_{d_2}$, $E_{d_3}$ are invarient along magnetic field lines, so these Apex components will be the same no matter what altitude they are mapped to.  To calculate the full vector, multiply each component by its corresponding base vector (Laundal and Richmond, 2016, eqn. 77 and 75).

$$ \vec{V_E} = v_{e_1}\bf{e_1} + v_{e_2}\bf{e_2} + v_{e_3}\bf{e_3} $$
$$ \vec{E} = E_{d_1}\bf{d_1} + E_{d_2}\bf{d_2} + E_{d_3}\bf{d_3} $$

The easiest way to calculate the base vectors is to use [apexpy](https://apexpy.readthedocs.io/en/latest/), a python package that provides efficient apex coordinate transformations.  The package also has a function which returns the apex base vectors at any given point with geodetic East, North, Up components.

In [36]:
from apexpy import Apex

# initialize an Apex object with the date
A = Apex(2019)

# specify the altitude (in km) to calculate the plasma drift velocity at
alt = 300.

# for simplicity, calculate the velocity field only for record 40
VE = Velocity[40,:,:]

First, caculate the location of each vector by convering the apex magnetic latitude and apex magnetic longitude to geodetic latitude and longitude at the specified altitude.

In [37]:
glat, glon, err = A.apex2geo(mlat, mlon, alt)
print 'Geodetic Latitude: ', glat
print 'Geodetic Longitude: ', glon

Geodetic Latitude:  [63.4838562  63.94165421 64.39841461 64.85412598 65.30880737 65.76244354
 66.21502686]
Geodetic Longitude:  [-148.40510559 -148.12579346 -147.83784485 -147.54084778 -147.23423767
 -146.91746521 -146.58992004]


Then, find the apex base vectors at each vector location at the specified altitude.  The `basevectors_apex` function returns the geodetic East, North, and Up components for both the quasi-dipole and the modified apex coordinate systems, but we will only be using those for modified apex. 

In [38]:
f1, f2, f3, g1, g2, g3, d1, d2, d3, e1, e2, e3 = A.basevectors_apex(mlat, mlon, alt, coords='apex')

For each bin, calculate the full vector with geodetic East, North, Up components.

In [39]:
for V, e01, e02, e03 in zip(VE, e1.T, e2.T, e3.T):
    VE_gd = V[0]*e01 + V[1]*e02 + V[2]*e03
    print VE_gd

[ 0.  0. -0.]
[ 0.  0. -0.]
[0.01982049 0.1800411  0.04308683]
[13.495016 29.186974  7.259793]
[1506.5571  -808.4749   -74.08618]
[1886.4762   -352.64023    43.018627]
[-2.2404764e-08 -2.9680290e-08 -7.4322690e-09]


### Using Einstein Summation
It may be more efficient to reform the base vectors into a rotation matrix and perform the operation with linear algebra functions.
$$ V_E = [e_1 e_2 e_3][v_{e_1} v_{e_2} v_{e_3}] $$
$$ V_E = [[e_1_E e_2_E e_3_E][e_1_N e_2_N e_3_N][e_1_Z e_2_Z e_3_Z][v_{e_1} v_{e_2} v_{e_3}] $$
$$ V_E = \sum e_{jk}V_{k} $$
The following example avoids `for` loops by using numpy's [einsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) function, and not summing over the bin index `i`.

In [40]:
import numpy as np
e = np.array([e1,e2,e3]).T
VE_gd = np.einsum('ijk,ik->ij',e,VE)
print VE_gd

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.98204930e-02  1.80041109e-01  4.30868328e-02]
 [ 1.34950160e+01  2.91869721e+01  7.25979234e+00]
 [ 1.50655700e+03 -8.08474923e+02 -7.40861852e+01]
 [ 1.88647624e+03 -3.52640288e+02  4.30186263e+01]
 [-2.24047638e-08 -2.96802913e-08 -7.43226915e-09]]


Furthermore, it is possible to use the einsum function to caculate the geodetic components of $V_E$ at all times.

In [41]:
Velocity_gd = np.einsum('ijk,...ik->...ij',e,Velocity)

The covariance matrix of geodetic velocity can also be found efficiently with linear algebra notation.
$$ \bf{\Sigma_{Vgd}} = \bf{e}\bf{\Sigma_V}\bf{e^T} $$

In [42]:
SigmaV_gd = np.einsum('ijk,...ikl,iml->...ijm',e,SigmaV,e)

As expected, the geodetic component velocity array and the apex component velocity array have the same shape.  The same is true for the covarience matricies.

In [43]:
print Velocity.shape, Velocity_gd.shape
print SigmaV.shape, SigmaV_gd.shape

(112, 7, 3) (112, 7, 3)
(112, 7, 3, 3) (112, 7, 3, 3)
